In [299]:
import cv2
import numpy as np

In [300]:
def display(image, scale=0.5):
    shrunk = cv2.resize(image, (0,0), fx=scale, fy=scale)
    cv2.imshow("Screen Location", shrunk)

    if cv2.waitKey(1000) & 0xFF == ord('q'):
        cv2.destroyAllWindows()

In [301]:
billboard_colour = [150, 20, 20]
def strip_colours(image, preserve=billboard_colour):
    black_indices = np.where((image != preserve).all(axis=2))
    white_indices = np.where((image == preserve).all(axis=2))
    rtn = image.copy()
    rtn[black_indices] = [0,0,0]
    rtn[white_indices] = [255,255,255]
    return rtn

# noinspection PyDefaultArgument
def add_borders(image, border_color= [0,0,0], border_width=25):
    return cv2.copyMakeBorder(
        image,
        border_width,
        border_width,
        border_width,
        border_width,
        cv2.BORDER_CONSTANT,
        value=border_color
    )

def remove_borders(image, border_width=25):
  return image[border_width:-border_width, border_width:-border_width]

def get_largest_billboard_contour(image):
    edged_image = cv2.Canny(binary_image, 30, 200)

    (contours, _) = cv2.findContours(
        edged_image.copy(),
        cv2.RETR_TREE,
        cv2.CHAIN_APPROX_SIMPLE
    )
    contours = sorted(
        contours,
        key=cv2.contourArea,
        reverse=True
    )[:10]


    best = (0,0)

    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        area = cv2.contourArea(contour)
        print(area, end="\t")
        print(len(approx))
        # if len(approx) == 4:
        if area > best[0]:
          best = (area, approx)

    return best[1] if best[0] > 0 else None

In [302]:
source_image = cv2.imread("../resources/new/015_00010.png")
print(source_image.shape)

padded_source_image = add_borders(source_image)
binary_image = strip_colours(padded_source_image)

(1080, 1920, 3)


In [303]:
display_image = padded_source_image.copy()
# Take the best (largest) contour
contour = get_largest_billboard_contour(binary_image)
cv2.drawContours(
    display_image,
    [contour],
    -1,
    (0,255,0),
    3
)

37642.5	4
37641.5	4
2413.5	4
2406.5	4
100.5	8
100.0	8
16.0	12
13.0	11
10.0	6
10.0	8


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [304]:
# crop out border we added earlier
display_image = remove_borders(display_image)

display(source_image)
display(display_image)

In [324]:
decal_image = cv2.imread("../resources/new/decal_0.png")

src = np.float32([
            [0,0],
            [decal_image.shape[0], 0],
            decal_image.shape[:2],
            [0, decal_image.shape[1]],
          ])

print(decal_image.shape)
print(src.shape)
print(contour.shape)
print(contour)
transform = cv2.getPerspectiveTransform(
    src,
    contour
)
print(transform)


(400, 600, 3)
(4, 2)
(8, 1, 2)
[[[1885  321]]

 [[1886  320]]

 [[1888  320]]

 [[1889  321]]

 [[1889  322]]

 [[1888  323]]

 [[1886  323]]

 [[1885  322]]]


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wbmte9m7\opencv\modules\imgproc\src\imgwarp.cpp:3391: error: (-215:Assertion failed) src.checkVector(2, CV_32F) == 4 && dst.checkVector(2, CV_32F) == 4 in function 'cv::getPerspectiveTransform'
